# Task 2. Deep Power Analysis attack

### created: 31.01.2020

### by David Zashkolny
### 3 course, comp math
### Taras Shevchenko National University of Kyiv
### email: davendiy@gmail.com


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, BatchNormalization
from keras.layers import Dropout, Activation, GlobalMaxPooling1D
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import pickle

## Preparing

In [4]:
# subtitution box for AES-128
Sbox = np.array(
       [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
)

# array of values of hamming_weight function for bytes from 0 to 255
hamming_weight = np.array([bin(n).count("1") for n in range(0, 256)])


def aes_first_round(data, key):
    """ Vectorized function for doing the first round of AES-128.
    """
    return hamming_weight[Sbox[data ^ key]]


def vcorrcoef(X, y):
    """ Vectorized function for Pearson's correlation coefficient.
    
    :param X: matrix (ndarray) of NxM elements
    :param y: vector (ndarray) of 1xM elements
    :return: vector C = [cor(X[0], y), cor(X[1], y), cor(X[2], y) ...]
    """
    Xm = np.reshape(np.mean(X,axis=1),(X.shape[0],1))
    ym = np.mean(y)
    r_num = np.sum((X-Xm)*(y-ym),axis=1)
    r_den = np.sqrt(np.sum((X-Xm)**2,axis=1)*np.sum((y-ym)**2))
    r = r_num/r_den
    return abs(r)

## Data preparing

In [5]:
train_textin = np.load('sca-f3/train/textin.npy')
train_traces = np.load('sca-f3/train/traces.npy')
train_keylist = np.load('sca-f3/train/keylist.npy')

y_train = aes_first_round(train_textin, train_keylist)   # required hamming weights
np.save('sca-f3/train/y_train.npy', y_train)

In [6]:
test_textin = np.load('sca-f3/test/textin.npy')
test_traces = np.load('sca-f3/test/traces.npy')
test_keylist = np.load('sca-f3/test/keylist.npy')
test_keylist = np.array([list(el) for el in test_keylist])
y_test = aes_first_round(test_textin, test_keylist)        # required hamming weights
np.save('sca-f3/test/y_test.npy', y_test)

In [7]:
coeffs = np.arange(30000)         # array of random coefficients
np.random.shuffle(coeffs)         # used for shuffling both x and y in same way
x = train_traces[coeffs[:27000]]
y = y_train[coeffs[:27000]]

x_val = train_traces[coeffs[27000:]]
y_val = y_train[coeffs[27000:]]

with open("sca-f3/test_data2.dav", 'wb') as file:    # data for colab 
    pickle.dump(((x, y), (x_val, y_train[coeffs[27000:]])), file)


## **Here you have to look at the task2_colab.ipynb

## Checking models

In [8]:
def create_model(path_to_weights=''):
    model = Sequential()
    model.add(Conv1D(5,          # filters
                     4,          # kernel_size
                     padding='valid', 
                     activation='relu', 
                     strides=1,
                     input_shape=(5000, 1))
             )
    model.add(BatchNormalization())
    model.add(Conv1D(5, 
                     10, 
                     padding='valid', 
                     activation='relu', 
                     strides=1)
             )
    model.add(BatchNormalization())
    model.add(Conv1D(3, 
                     15, 
                     padding='same', 
                     activation='relu', 
                     strides=10)
             )
    model.add(BatchNormalization())
    model.add(Conv1D(2, 
                     15, 
                     padding='same', 
                     activation='relu', 
                     strides=10)
             )
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(9, activation='softmax'))
    
    if path_to_weights:
        model.load_weights(path_to_weights)
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adadelta', 
                  metrics=['accuracy'])
    return model


test_model = create_model('sca-f3/models/byte11/weights-improvement-19-0.87.hdf5')
test_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 4997, 5)           25        
_________________________________________________________________
batch_normalization_1 (Batch (None, 4997, 5)           20        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4988, 5)           255       
_________________________________________________________________
batch_normalization_2 (Batch (None, 4988, 5)           20        
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 499, 3)            228       
_________________________________________________________________
batch_normalization_3 (Batch (None, 499, 3)            12        
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 50, 2)            

### Let's load all the models we trained on colab

In [26]:
names = {
    0:  "weights-improvement-94-0.77.hdf5",      # <byte_num> : <path> 
    1:  "weights-improvement-85-0.77.hdf5",      # weights-improvement-{epoch}-{val_acc}
    2:  "weights-improvement-93-0.84.hdf5",
    3:  "weights-improvement-100-0.81.hdf5",
    4:  "weights-improvement-85-0.72.hdf5",
    5:  "weights-improvement-94-0.96.hdf5",
    6:  "weights-improvement-29-0.81.hdf5",
    7:  "weights-improvement-91-0.87.hdf5",
    8:  "weights-improvement-98-0.71.hdf5",
    9:  "weights-improvement-92-0.84.hdf5",
    10: "weights-improvement-97-0.83.hdf5",
    11: "weights-improvement-89-0.91.hdf5",
    12: "weights-improvement-100-0.70.hdf5",
    13: "weights-improvement-94-0.78.hdf5",
    14: "weights-improvement-94-0.86.hdf5",
    15: "weights-improvement-29-0.84.hdf5",
}

models = [create_model(f'sca-f3/models/byte{i}/{names[i]}') for i in range(16)]

In [27]:
x = test_traces[:1000].reshape(1000, 5000, 1)

res_losses = []
res_accs = []
for i in range(16):
    y = to_categorical(y_test[:, i])
    cur_loss, cur_acc = models[i].evaluate(x, y)
    res_losses.append(cur_loss)
    res_accs.append(cur_acc)
print("accuracy for test data:", res_accs)
print("\nloss for test data:", res_losses)

1000/1000 [==============================] - 2s 2ms/step
accuracy for test data: [0.7639999985694885, 0.75, 0.824999988079071, 0.7870000004768372, 0.7099999785423279, 0.953000009059906, 0.800000011920929, 0.8550000190734863, 0.7120000123977661, 0.8429999947547913, 0.8420000076293945, 0.9240000247955322, 0.6639999747276306, 0.7979999780654907, 0.8560000061988831, 0.8299999833106995]

loss for test data: [0.5376956882476807, 0.5387540893554688, 0.44922966861724856, 0.4628621196746826, 0.6658350563049317, 0.13618068194389343, 0.4947180528640747, 0.3794122357368469, 0.6614728655815124, 0.38310437881946563, 0.4006687879562378, 0.21742523169517516, 0.742246563911438, 0.5050541687011719, 0.3687541944980621, 0.4386028051376343]


### Intermed

I think, the result is good. The accuracy not so big as on validation data, but it's not as much, as it could be.

In [28]:
print("average:", np.mean(res_accs))
print(f"min accuracy: {np.min(res_accs)} for {np.argmin(res_accs)} byte")
print(f"max accuracy: {np.max(res_accs)} for {np.argmax(res_accs)} byte")

average: 0.8070624992251396
min accuracy: 0.6639999747276306 for 12 byte
max accuracy: 0.953000009059906 for 5 byte


In [12]:
def investigate(text_in, traces, models, true_key, printt=True):
    
    # array 256xNx16 = <possible_bytes> x <amount_of_blocks> x <key's_len>
    hypotheses = np.array([np.zeros((len(text_in), 16), 'uint8') + i for i in range(256)])
    
    # hws[hyp, sample_num, byte_num] = hamming weight of <byte_num> byte of <sample_num> block
    # after making the first round of AES using the <byte_num> byte of key that equals to <hyp>
    hws = aes_first_round(text_in, hypotheses)
    
    if printt:
        print(f"true_key: {true_key}")

    ta_l = 1               # initial parameters for binary search
    ta_r = len(traces)     # corresponding left and right bounds

    traces_amount = len(traces)
    
    # Models predict hamming weights for the given traces
    predicted_hws = [np.argmax(models[i].predict(traces.reshape(*traces.shape, 1)), axis=1) for i in range(16)]
    predicted_hws = np.array(predicted_hws)
    
    # binary search
    while True:
        traces_amount = ta_l + (ta_r - ta_l) // 2   # current step 
        if (ta_r - ta_l) < 1:
             traces_amount = ta_r
             break
            
        if printt:
            print(f'\ntraces_amount: {traces_amount}')
            
        # initial parameters for DPA
        best_corrs = np.zeros(16)            # array of best correlations for each byte of key
        best_bytes = np.zeros(16, 'uint8')   # array of best values for each byte of key
         
        for byte_num in range(16):
            hw = hws[:, :traces_amount, byte_num]   # corresponding hamming weights
            cur_cors = vcorrcoef(hw, predicted_hws[byte_num, :traces_amount])   # calculates the correlations
            best_corrs[byte_num] = np.max(cur_cors)         # find the hyp with max correlation
            best_bytes[byte_num] = np.argmax(cur_cors)    
            
        # step of binary search
        # if we can rebuild the key - we must take the less number of traces
        # otherwise - more number
        if not any(best_bytes != true_key):   
            ta_r = traces_amount                            
        else:
            ta_l = traces_amount + 1
            
        if printt:
#             print(f'\nfound_key: {best_bytes}')
#             print(f'correlations: {best_corrs}')
            print(f'bad bytes: {np.arange(16)[best_bytes != true_key]}')
    return traces_amount

In [118]:
res_mins = []
for i in range(0, len(test_textin)//100):
    
    tmp_res = investigate(test_textin[i*100: (i+1)*100], 
                          test_traces[i*100: (i+1)*100], 
                          models, test_keylist[i*100])
    res_mins.append(tmp_res)

true_key: [117  78  52  68  90  68 107 115 110  82 113 111 115  49  49 103]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 2  4 10 12]

traces_amount: 10
bad bytes: [4]

traces_amount: 12
bad bytes: []

traces_amount: 11
bad bytes: []
true_key: [ 98 117  85  76  90 111 120  87  98  84  85 114  78 120 107  82]


/home/davidas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide



traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [7]

traces_amount: 10
bad bytes: []

traces_amount: 9
bad bytes: []

traces_amount: 8
bad bytes: []
true_key: [115  52  83  65 122 109  84 113 109  68  53  53  52 117 115  66]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 0  4  8  9 12]

traces_amount: 10
bad bytes: [8]

traces_amount: 12
bad bytes: [8]
true_key: [ 89 118  74 101  98  99  85 109  97  84  48  65 109  89  77 105]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 0  2  4  6 12 15]

traces_amount: 10
bad bytes: []

traces_amount: 9
bad bytes: [0 2]
true_key: [ 86  88 101 122 118 101  56  90  98 121 101  86  52  49  51  69]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount:

In [123]:
res_mins = np.array(res_mins)
print("minimum traces for correct rebuilding of key:", res_mins)
print("max:", np.max(res_mins), "average:", res_mins.mean())

minimum traces for correct rebuilding of key: [11  8 13 10 13 11 11 14 13 11]
max: 14 average: 11.5


# Bonus

Now let's check retrained models. I took all the best modesl from previous part, 
and made an extra fit with 150 epoches.

In [10]:
names2 = {
    0:  "weights-improvement-131-0.79.hdf5",   # <byte_num> : <path> 
    1:  "weights-improvement-134-0.81.hdf5",   # weights-improvement-{epoch}-{val_acc}
    2:  "weights-improvement-127-0.87.hdf5",
    3:  "weights-improvement-143-0.86.hdf5",
    4:  "weights-improvement-99-0.75.hdf5",
    5:  "weights-improvement-131-0.98.hdf5",   # 98%, not bad
    6:  "weights-improvement-132-0.86.hdf5",
    7:  "weights-improvement-140-0.90.hdf5",
    8:  "weights-improvement-147-0.73.hdf5",
    9:  "weights-improvement-94-0.86.hdf5",
    10: "weights-improvement-147-0.85.hdf5",
    11: "weights-improvement-138-0.93.hdf5",
    12: "weights-improvement-138-0.73.hdf5",
    13: "weights-improvement-146-0.82.hdf5",
    14: "weights-improvement-115-0.91.hdf5",
    15: "weights-improvement-136-0.90.hdf5",
    
}

models2 = [create_model(f'sca-f3/models_100_plus_add/byte{i}/{names2[i]}') for i in range(16)]

### Checking the accuracy on test data (models didn't use it during training)

In [20]:
x = test_traces[:1000].reshape(1000, 5000, 1)

res_losses = []
res_accs = []
for i in range(16):
    y = to_categorical(y_test[:, i])
    cur_loss, cur_acc = models2[i].evaluate(x, y)
    res_losses.append(cur_loss)
    res_accs.append(cur_acc)
print("accuracy for test data:", res_accs)
print("loss for test data:", res_losses)

1000/1000 [==============================] - 2s 2ms/step
accuracy for test data: [0.7770000100135803, 0.7710000276565552, 0.8410000205039978, 0.8379999995231628, 0.7289999723434448, 0.968999981880188, 0.8389999866485596, 0.8769999742507935, 0.7279999852180481, 0.843999981880188, 0.8450000286102295, 0.9380000233650208, 0.6800000071525574, 0.8389999866485596, 0.8989999890327454, 0.9070000052452087]
loss for test data: [0.5290756545066834, 0.48385641956329345, 0.38230846738815305, 0.37339052867889405, 0.6321179304122925, 0.09304169153049588, 0.4013866147994995, 0.2955913541316986, 0.639751838684082, 0.385775710105896, 0.38158990907669066, 0.18268015146255492, 0.683857313156128, 0.4210977442264557, 0.25839447593688963, 0.2768600304722786]


In [24]:
print("average:", np.mean(res_accs))
print(f"min accuracy: {np.min(res_accs)} for {np.argmin(res_accs)} byte")
print(f"max accuracy: {np.max(res_accs)} for {np.argmax(res_accs)} byte")

average: 0.8325624987483025
min accuracy: 0.6800000071525574 for 12 byte
max accuracy: 0.968999981880188 for 5 byte


### In fact the grows isn't as big on test data as we saw at validation data.
### Let's check the key rebuilding

In [14]:
res_mins = []
for i in range(0, len(test_textin)//100):
    
    tmp_res = investigate(test_textin[i*100: (i+1)*100], 
                          test_traces[i*100: (i+1)*100], 
                          models2, test_keylist[i*100])
    res_mins.append(tmp_res)

true_key: [117  78  52  68  90  68 107 115 110  82 113 111 115  49  49 103]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 2  4 12]

traces_amount: 10
bad bytes: [4]

traces_amount: 12
bad bytes: []

traces_amount: 11
bad bytes: []
true_key: [ 98 117  85  76  90 111 120  87  98  84  85 114  78 120 107  82]


/home/davidas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide



traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [4 7]

traces_amount: 10
bad bytes: []

traces_amount: 9
bad bytes: []

traces_amount: 8
bad bytes: []
true_key: [115  52  83  65 122 109  84 113 109  68  53  53  52 117 115  66]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 0  8  9 12]

traces_amount: 10
bad bytes: []

traces_amount: 9
bad bytes: []

traces_amount: 8
bad bytes: [12]
true_key: [ 89 118  74 101  98  99  85 109  97  84  48  65 109  89  77 105]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad bytes: []

traces_amount: 7
bad bytes: [ 0  2  4 15]

traces_amount: 10
bad bytes: []

traces_amount: 9
bad bytes: [0]
true_key: [ 86  88 101 122 118 101  56  90  98 121 101  86  52  49  51  69]

traces_amount: 50
bad bytes: []

traces_amount: 25
bad bytes: []

traces_amount: 13
bad byt

### Conclusion

As we can see, retraining was not so effective. Moreover, the the average number of traces didn't change, but the maximum now is 18 (instead of 14 in previous models).

Taking into account the fact that I spent over 3 hours on the retraining (instead of 1.5 hour on the first), I think that it was just a waste of time

In [15]:
res_mins = np.array(res_mins)
print("minimum traces for correct rebuilding of key:", res_mins)
print("max:", np.max(res_mins), "average:", res_mins.mean())

minimum traces for correct rebuilding of key: [11  8  9 10 17  7 11 18 15  9]
max: 18 average: 11.5
